In [1]:
#Insert Description stuff

# Exploratory Data Analysis 

In [2]:
#Imports for EDA
import pandas as pd
from pandas_profiling import ProfileReport

## Read Dataset

In [3]:
date_format = '%d/%m/%y'
# Read all the dataset
#Campaign Data
campaign_data_df = pd.read_csv('campaign_data.csv')
campaign_data_df.start_date = pd.to_datetime(campaign_data_df.start_date,format=date_format)
campaign_data_df.end_date = pd.to_datetime(campaign_data_df.end_date,format=date_format)
# Customer Demographics
customer_demographics_df = pd.read_csv('customer_demographics.csv')
# Train Data
train_df = pd.read_csv('train.csv')
#Read customer transaction data
customer_transaction_data_df = pd.read_csv('customer_transaction_data/customer_transaction_data.csv')
customer_transaction_data_df.date = pd.to_datetime(customer_transaction_data_df.date) #Convert to datetime

#Read Item Data
item_data_df = pd.read_csv('item_data.csv')
#Coupon item mapping
coupon_item_mapping_df = pd.read_csv('coupon_item_mapping.csv')

## Combine Similar Dataframes

In [5]:
# Merge the 3 dataframes
#df1 - train dataset combined with the campaign data to get the details on campaign 
#       e.g. campaign_type, start and end date of the campaign
df1 = train_df.merge(campaign_data_df,on='campaign_id',how='left').merge(customer_demographics_df,on='customer_id',how='left')
df1.head()


,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,26543,23,1081,865,0,Y,2012-10-08,2012-11-30,26-35,NaN,0.0,1,NaN,7.0
1,432,30,108,49,0,Y,2013-04-22,2013-05-24,NaN,NaN,NaN,NaN,NaN,NaN
2,59998,6,773,1582,0,X,2013-05-19,2013-07-05,NaN,NaN,NaN,NaN,NaN,NaN
3,127582,28,82,521,0,X,2013-02-16,2013-04-05,NaN,NaN,NaN,NaN,NaN,NaN
4,61568,21,666,1462,0,X,2012-11-19,2013-01-04,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#df2 - transaction data merged with the item data to get the item details
#  e.g. brand, brand_type and category
df2 = customer_transaction_data_df.merge(item_data_df,on='item_id',how='left')
df2.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0,56,Local,Natural Products
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0,56,Local,Natural Products
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0,524,Established,Grocery
3,2012-01-02,1501,33647,1,67.32,0.00,0.0,1134,Established,Grocery
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0,524,Established,Grocery


## Run Profile Report

In [ ]:
# Running Profile Report on df2 reveals that:
#     - columns 'quantity' and selling price have high standard deviation, meaning there are outliers
#     - some of the customer transactions are duplicated    

## Remove Duplicates

In [13]:
df2.drop_duplicates(inplace=True)
df2

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0,56,Local,Natural Products
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0,56,Local,Natural Products
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0,524,Established,Grocery
3,2012-01-02,1501,33647,1,67.32,0.00,0.0,1134,Established,Grocery
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0,524,Established,Grocery
...,...,...,...,...,...,...,...,...,...,...
1324561,2013-06-30,1129,2777,1,284.60,-71.24,0.0,2074,Established,Grocery
1324562,2013-06-30,1129,2953,4,42.74,-28.50,0.0,56,Local,Grocery
1324563,2013-06-30,1129,2971,6,64.12,-42.74,0.0,56,Local,Grocery
1324564,2013-06-30,1129,46984,1,95.82,0.00,0.0,56,Local,Grocery


## Remove Exteme Values

In [17]:
from scipy import stats
import numpy as np

In [21]:
outliers_df = df2[(np.abs(stats.zscore(df2[['quantity','selling_price']]))>4).all(axis=1)] 
outliers_df

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category
4917,2012-01-18,632,49009,11066,819.62,-39.18,0.0,56,Local,Fuel
6360,2012-01-20,503,49009,12025,890.50,-42.74,0.0,56,Local,Fuel
6807,2012-01-21,401,2862,11266,890.50,-12.11,0.0,56,Local,Fuel
6912,2012-01-22,415,49004,11524,890.50,-40.96,0.0,56,Local,Miscellaneous
7636,2012-01-23,225,49009,17795,1355.70,-0.36,0.0,56,Local,Fuel
...,...,...,...,...,...,...,...,...,...,...
1324351,2013-07-03,590,49009,21837,1554.81,-78.01,0.0,56,Local,Fuel
1324438,2013-07-03,1368,49009,17830,1282.32,-63.40,0.0,56,Local,Fuel
1324439,2013-07-03,1168,49009,12025,864.85,-42.74,0.0,56,Local,Fuel
1324444,2013-07-03,453,39453,19364,1496.04,-20.66,0.0,56,Local,Fuel


In [22]:
#Most of the outliers are in category 'Fuel'
outliers_df.category.value_counts()

Fuel             8524
Miscellaneous    1325
Name: category, dtype: int64

In [23]:
#All the outliers are brand 56 
outliers_df.brand.value_counts()

56    9849
Name: brand, dtype: int64

In [24]:
#Remove outliers
df2 = df2[(np.abs(stats.zscore(df2[['quantity','selling_price']]))<4).all(axis=1)] 

In [26]:
#Merge df2 to coupon_item_mapping_df to get coupon id (needed for aggregation later on)
df3 = df2.merge(coupon_item_mapping_df,on='item_id',how='left')
df3


,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category,coupon_id
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0,56,Local,Natural Products,7.0
1,2012-01-02,1501,26830,1,35.26,-10.69,0.0,56,Local,Natural Products,20.0
2,2012-01-02,1501,26830,1,35.26,-10.69,0.0,56,Local,Natural Products,29.0
3,2012-01-02,1501,54253,1,53.43,-13.89,0.0,56,Local,Natural Products,7.0
4,2012-01-02,1501,54253,1,53.43,-13.89,0.0,56,Local,Natural Products,20.0
...,...,...,...,...,...,...,...,...,...,...,...
2629121,2013-06-30,1129,2971,6,64.12,-42.74,0.0,56,Local,Grocery,NaN
2629122,2013-06-30,1129,46984,1,95.82,0.00,0.0,56,Local,Grocery,367.0
2629123,2013-06-30,1129,64498,2,489.78,0.00,0.0,3514,Established,Meat,8.0
2629124,2013-06-30,1129,64498,2,489.78,0.00,0.0,3514,Established,Meat,24.0


# Feature Engineering

## Coupon-Item Details

In [27]:
def add_coupon_details(x):
    """Function that assigns the value of 1.0 to columns if the value is in Dataframe by filtering using:
        -customer_id
        -coupon_id
        -whether date falls between the start and end date of campaign
        """
    df = df3[(df3.customer_id == x['customer_id'])&(df3.coupon_id == x['coupon_id'])]
    df = df[(df.date>=x.start_date)&(df.date<=x.end_date)]
    if len(df)>3:
        #get brand type and category
        df_cols = np.append(df.brand_type.unique(),df.category.unique())
        #put value in the column
        x[df_cols]=1.0
    return x

In [29]:
#Use unique values from the brand_type and category columns as new columns
new_cols = np.append(df3.brand_type.unique(),df3.category.unique())
df1[new_cols]=0
df1.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,age_range,marital_status,rented,family_size,no_of_children,income_bracket,Local,Established,Natural Products,Grocery,Meat,Packaged Meat,Pharmaceutical,Seafood,Prepared Food,"Dairy, Juices & Snacks",Bakery,Flowers & Plants,Alcohol,Skin & Hair Care,Miscellaneous,Garden,Vegetables (cut),Fuel,Salads,Restauarant,Travel
0,26543,23,1081,865,0,Y,2012-10-08,2012-11-30,26-35,NaN,0.0,1,NaN,7.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,432,30,108,49,0,Y,2013-04-22,2013-05-24,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,59998,6,773,1582,0,X,2013-05-19,2013-07-05,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,127582,28,82,521,0,X,2013-02-16,2013-04-05,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,61568,21,666,1462,0,X,2012-11-19,2013-01-04,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
df1 = df1.apply(lambda x:add_coupon_details(x),axis=1)

## Aggregation of Numerical Values

In [31]:
from tqdm import tqdm

In [32]:
#Different statistical methods and corresponding columns where the methods will be applied
calc_dict = {'mean':np.mean,
            'sum':np.sum,
            'std':np.std,
            'min':np.min,
            'max':np.max}
cols_list = ['selling_price','coupon_discount','other_discount','quantity']

In [33]:
def calculate(x):
    """Generate features"""
    df = df3[(df3.customer_id == x['customer_id'])&(df3.coupon_id == x['coupon_id'])]
    df = df[(df.date>=x.start_date)&(df.date<=x.end_date)]
    out_list = []
    #Calculate stuff 
    for calc in calc_dict:
        method = calc_dict[calc]
        for col in cols_list:
            out_list.append(df[col].aggregate(method))
    return out_list

In [34]:
#Aggregate
x = []
for _,row in tqdm(df1.iterrows(),total=df1.shape[0]):
    x.append(calculate(row))
x1 = np.array(x).T

100%|██████████| 77157/77157 [14:58<00:00, 85.90it/s]


In [35]:
#Assign the columns
counter = 0
for calc in calc_dict:
    method = calc_dict[calc]
    for col in cols_list:
        df1[f'{calc}_{col}'] = x1[counter]
        counter+=1
df1.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,age_range,marital_status,rented,family_size,no_of_children,income_bracket,Local,Established,Natural Products,Grocery,Meat,Packaged Meat,Pharmaceutical,Seafood,Prepared Food,"Dairy, Juices & Snacks",Bakery,Flowers & Plants,Alcohol,Skin & Hair Care,Miscellaneous,Garden,Vegetables (cut),Fuel,Salads,Restauarant,Travel,mean_selling_price,mean_coupon_discount,mean_other_discount,mean_quantity,sum_selling_price,sum_coupon_discount,sum_other_discount,sum_quantity,std_selling_price,std_coupon_discount,std_other_discount,std_quantity,min_selling_price,min_coupon_discount,min_other_discount,min_quantity,max_selling_price,max_coupon_discount,max_other_discount,max_quantity
0,26543,23,1081,865,0,Y,2012-10-08,2012-11-30,26-35,NaN,0.0,1,NaN,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,432,30,108,49,0,Y,2013-04-22,2013-05-24,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,59998,6,773,1582,0,X,2013-05-19,2013-07-05,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,127582,28,82,521,0,X,2013-02-16,2013-04-05,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,61568,21,666,1462,0,X,2012-11-19,2013-01-04,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Campaign length

In [36]:
df1['campaign_days'] = (df1.end_date - df1.start_date).dt.days

## One hot encoding of Categorical Features

In [37]:
from sklearn.preprocessing import OneHotEncoder

In [38]:
#Onehot transform the campaign_type
enc = OneHotEncoder()
to_encode = np.array(df1.campaign_type).reshape(-1,1)
enc_fit = enc.fit(to_encode)
enc_transformed = enc.transform(to_encode).toarray()
df1[enc.categories_[0]] = enc_transformed
df1.drop(columns=['campaign_type'],inplace=True)

In [39]:
#Onehot transform the marital_status
enc = OneHotEncoder()
to_encode = np.array(df1.marital_status).reshape(-1,1)
enc_fit = enc.fit(to_encode)
enc_transformed = enc.transform(to_encode).toarray()
df1[enc.categories_[0]] = enc_transformed
df1.drop(columns=['marital_status',np.NaN],inplace=True)

## Reassign values

In [40]:
# Change age
age_mapping = { '18-25':20.0,
                '26-35':30.0,
                '36-45':40.0,
                '46-55':50.0,
                '56-70':60.0,
                '70+':70
                }
df1.age_range = df1.age_range.map(age_mapping)

In [41]:
#Change family size and no_of_children
def convert_to_int(x):
    try:
        out =  int(re.findall(r'^\D*(\d+)', str(x))[0])
    except:
        out = np.nan
    return out
df1.family_size = df1.family_size.apply(lambda x:convert_to_int(x))
df1.no_of_children = df1.no_of_children.apply(lambda x:convert_to_int(x))

In [42]:
#Drop columns that are not needed
df1.drop(columns=['start_date','end_date','id','campaign_id','coupon_id','customer_id'],inplace=True)

# Training

## Prepare Dataset

In [44]:
from sklearn.model_selection import train_test_split

In [43]:
#Get X and y
target_col_name = 'redemption_status' 
y = df1[target_col_name]
X = df1.drop(target_col_name,axis=1)

In [45]:
# Separate Validation Data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

## Training

In [46]:
from xgboost import XGBClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
import joblib

In [47]:
#Use XGboost Classifier
model = XGBClassifier(use_label_encoder=False,eval_metric='logloss')

In [62]:
#Set the grid for grid search
param_grid = {'scale_pos_weight':[3,5,10],
            'eta':[0.1,0.2,0.3],
            'gamma':[2,5],
            'max_depth':[6,10,15],
            'n_estimators' :[100,300]}

In [65]:
#Train
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='f1_macro',verbose=2)
grid_result = grid.fit(np.array(X_train),np.array(y_train))
#Output the model
joblib.dump(grid_result, 'model1.pkl')

Fitting 15 folds for each of 108 candidates, totalling 1620 fits


In [55]:
print("Best f1_score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best f1_score: 0.707796 using {'eta': 0.1, 'gamma': 2, 'max_depth': 10, 'n_estimators': 300, 'scale_pos_weight': 3}


## Validation

In [60]:
from sklearn.metrics import classification_report

In [57]:
#Get prediction of validation data
y_pred = grid_result.best_estimator_.predict(X_test)

In [61]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99     22514
           1       0.48      0.32      0.38       634

    accuracy                           0.97     23148
   macro avg       0.73      0.66      0.68     23148
weighted avg       0.97      0.97      0.97     23148



# Possible Future Improvements

1. Use different classifiers
    - Neural Networks
    - SVM
    - Decision Tree
    - Random Forest
2. Use Ensemble, combining the different classifiers
3. Get feature importance to get rid of low impact features
4. Balance the datasets 